# Taking Apart flux.fits

In [1]:
from astropy.io import fits
from astropy import table
from astropy import units as u
from astropy import constants as const
import numpy as np


In [2]:
# Looking into an example model...let's choose 's-p-hmi' and start with 'flux.fits'
hdulflux = fits.open('../data/YSOmodels/s-p-hmi/flux.fits')
hdulflux.info()

Filename: ../data/YSOmodels/s-p-hmi/flux.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       8   (10000,)   int64   
  1  MODEL_NAMES    1 BinTableHDU     11   10000R x 1C   [20A]   
  2  SPECTRAL_INFO    1 BinTableHDU     15   200R x 2C   [D, D]   
  3  APERTURES     1 BinTableHDU     12   20R x 1C   [D]   
  4  VALUES        1 ImageHDU        10   (200, 20, 10000)   float32   
  5  UNCERTAINTIES    1 ImageHDU        10   (200, 20, 10000)   float32   


In [3]:
hdulflux[0].header #Shows breakdown of the sections, SEDCube reads in the assumed distance from this

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                   64 / array data type                                
NAXIS   =                    1 / number of array dimensions                     
NAXIS1  =                10000                                                  
EXTEND  =                    T                                                  
DISTANCE=       3.08568025E+21 / Distance assumed for the values, in cm         
NWAV    =                  200 / Number of wavelengths                          
NAP     =                   20 / Number of apertures                            

In [4]:
hdulflux[0].data #Validity array, 0 for invalid, 1 for valid, about 90% of points are valid

array([1, 0, 1, ..., 1, 1, 1], dtype='>i8')

In [5]:
hdulflux['MODEL_NAMES'].header 
#'20A' is a sorting notation that says the table is sorted by the first 20 ascii characters in ascending order

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   20 / length of dimension 1                          
NAXIS2  =                10000 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    1 / number of table fields                         
TTYPE1  = 'MODEL_NAME'                                                          
TFORM1  = '20A     '                                                            
EXTNAME = 'MODEL_NAMES'        / extension name                                 

In [7]:
hdulflux['MODEL_NAMES'].data 
#List of the model names in this set, weirdly is a tuple, but SEDCube only takes the 'MODEL_NAME' half

FITS_rec([('001fS22r_01',), ('00adoafo_01',), ('00obfcP6_01',), ...,
          ('zZtw8Idg_01',), ('zz4i8Bbo_01',), ('zzasIb8I_01',)],
         dtype=(numpy.record, [('MODEL_NAME', 'S20')]))

In [9]:
print(hdulflux['SPECTRAL_INFO'].header)
print(hdulflux['SPECTRAL_INFO'].data) #Tuple of Wavelength (um) and Frequency (Hz) under names 'WAVELENGTH' and 'FREQUENCY'

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                   16 / length of dimension 1                          NAXIS2  =                  200 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    2 / number of table fields                         TTYPE1  = 'WAVELENGTH'                                                          TFORM1  = 'D       '                                                            TTYPE2  = 'FREQUENCY'                                                           TFORM2  = 'D       '                                                            EXTNAME = 'SPECTRAL_INFO'      / extensi

In [23]:
print(repr(hdulflux['APERTURES'].header)) #Aperture sizes in units of (AU), also in a tuple with empty value
print(hdulflux['APERTURES'].data)

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                    8 / length of dimension 1                          
NAXIS2  =                   20 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    1 / number of table fields                         
TTYPE1  = 'APERTURE'                                                            
TFORM1  = 'D       '                                                            
EXTNAME = 'APERTURES'          / extension name                                 
TUNIT1  = 'AU      '                                                            
[(1.00000000e+02,) (1.623776

In [2]:
with fits.open('../data/YSOmodels/s-p-hmi/flux.fits') as hdulflux:
    print(repr(hdulflux['UNCERTAINTIES'].header))
    print(repr(hdulflux['UNCERTAINTIES'].data))

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                   20                                                  
NAXIS3  =                10000                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
BUNIT   = 'mJy     '                                                            
EXTNAME = 'UNCERTAINTIES'      / extension name                                 
array([[[1.99267101e-02, 2.53303777e-02, 3.21958028e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.17076780e-02, 4.03170288e-02, 5.12592420e-02, ...,
        

## Creating a new Model from scratch ('b-s')

In [20]:
#data = np.arange(100.0) # a simple sequence of floats from 0.0 to 99.0
#hdu = fits.PrimaryHDU(data=data) # creates a primary HDU object (header data unit)
#hdul = fits.HDUList([hdu]) # creates an hdulist object containing primary hdu and could have others
#hdul.writeto('new1.fits') # writes this to a fits file -- THIS ONLY WORKS FOR IMAGE FITS
#--------------------#
a1 = np.zeros(10000)
for i in range(1,10000):
    a1[i] = np.random.choice([0,1],p=[.1,.9])
a1 = a1.astype(int)
primhead = fits.Header([fits.Card('DISTANCE',3E+21,'Distance in cm'),
                        fits.Card('NWAV',200,'Number of wavelengths'),fits.Card('NAP',20,'Number of Apertures')])
primhdu = fits.PrimaryHDU(a1,primhead)

modlist = np.arange(0,10000,1)
modlist.astype(str)
col1 = fits.Column(name='MODEL_NAME', format='20A', array=modlist) #Column objects package the arrays with the typical fits info: name and format
cols = fits.ColDefs([col1]) #ColDefs object packages the column objects
modhdu = fits.BinTableHDU.from_columns(cols,name = 'MODEL_NAMES') #Creates the bintablehdu object to be made to FITS_rec

speclist = np.logspace(-2,3.7,200)
freqlist = const.c.value / speclist
speccol1 = fits.Column(name='WAVELENGTH',format = 'D', unit = 'um', array=speclist)
speccol2 = fits.Column(name='FREQUENCY', format = 'D', unit = 'Hz', array=freqlist)
speccols = fits.ColDefs([speccol1,speccol2])
spechdu = fits.BinTableHDU.from_columns(speccols,name = 'SPECTRAL_INFO')

aplist = np.logspace(2,6,20)
apcol1 = fits.Column(name='APERTURE', format = 'D', unit = 'AU', array=aplist)
apcols = fits.ColDefs([apcol1])
aphdu = fits.BinTableHDU.from_columns(apcols,name = 'APERTURES')

valdata = np.zeros((10000,20,200))
valdata += np.random.normal(10**3,10*2,(10000,20,200))
valhdr = fits.Header([fits.Card('BUNIT','mJy','')])
valhdu = fits.ImageHDU(data=valdata,name='VALUES',header = valhdr)

uncdata = np.zeros((10000,20,200))
uncdata += np.absolute(np.random.normal(0,10**-2,(10000,20,200)))
unchdr = fits.Header([fits.Card('BUNIT','mJy','')])
unchdu = fits.ImageHDU(data=uncdata,name='UNCERTAINTIES',header = unchdr)


hdulist = fits.HDUList([primhdu,modhdu,spechdu,aphdu,valhdu,unchdu])
#OR
# hdu = fits.BinTableHDU.from_columns(
#     [fits.Column(name='target', format='20A', array=a1),
#      fits.Column(name='V_mag', format='E', array=a2)])
#
#OR Use table interface
#c1 = fits.Column(name='a', array=np.array([1, 2]), format='K')
#c2 = fits.Column(name='b', array=np.array([4, 5]), format='K')
#c3 = fits.Column(name='c', array=np.array([7, 8]), format='K')
#t = fits.BinTableHDU.from_columns([c1, c2, c3])
#t.writeto('table2.fits')

hdulist.writeto('../data/YSOmodels/b-s/flux.fits',overwrite=True)

In [21]:
with fits.open('../data/YSOmodels/b-s/flux.fits') as testfits:
    testfits.info()
    #print(repr(testfits[0].header))
    #print(repr(testfits[0].data))
    print(repr(testfits[5].header))
    print(repr(testfits[5].data))

Filename: ../data/YSOmodels/b-s/flux.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       8   (10000,)   int64   
  1  MODEL_NAMES    1 BinTableHDU     11   10000R x 1C   [20A]   
  2  SPECTRAL_INFO    1 BinTableHDU     15   200R x 2C   [D, D]   
  3  APERTURES     1 BinTableHDU     12   20R x 1C   [D]   
  4  VALUES        1 ImageHDU        10   (200, 20, 10000)   float64   
  5  UNCERTAINTIES    1 ImageHDU        10   (200, 20, 10000)   float64   
XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -64 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                   20                                                  
NAXIS3  =                10000                                                  
PCOUNT  

# Parameter Files, etc.

In [3]:
# 'parameters.fits'
hdulparam = fits.open('../data/YSOmodels/s-p-hmi/parameters.fits')
hdulparam.info()

Filename: ../data/YSOmodels/s-p-hmi/parameters.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     28   10000R x 10C   [20A, D, D, D, D, D, D, D, K, D]   


In [7]:
hdulstel = fits.open('../data/YSOmodels/s-p-hmi/stellar.fits')
hdulstel.info()

Filename: ../data/YSOmodels/s-p-hmi/stellar.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   (10000,)   int64   
  1  MODEL_NAMES    1 BinTableHDU     11   10000R x 1C   [20A]   
  2  SPECTRAL_INFO    1 BinTableHDU     15   200R x 2C   [D, D]   
  3  VALUES        1 ImageHDU        10   (200, 1, 10000)   float32   


# Convolved Fluxes

In [24]:
with fits.open('../data/YSOmodels/s-p-hmi/convolved/W2.fits') as convfits:
    convfits.info()
    print(repr(convfits[0].header))
    #print(repr(convfits[1].header))
    #print(repr(convfits[2].header))

Filename: ../data/YSOmodels/s-p-hmi/convolved/W2.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1  CONVOLVED FLUXES    1 BinTableHDU     17   10000R x 3C   [30A, 20D, 20D]   
  2  APERTURES     1 BinTableHDU     12   20R x 1C   [D]   
SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
FILTWAV =                863.0                                                  
NMODELS =                10000                                                  
NAP     =                   20                                                  


# Fits Closer

In [ ]:
hdulflux.close()
hdulparam.close()
hdulstel.close()

In [9]:
%cat ../data/YSOmodels/s-p-hmi/models.conf

name = s-p-hmi
# Robitaille (2017) YSO SED models v1.1
# https://doi.org/10.5281/zenodo.166732
length_subdir = 2
aperture_dependent = yes
logd_step = 0.02
version = 2
